In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-2mlyjyjq
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-2mlyjyjq
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu
#include <iostream>
#include <stdlib.h>

using namespace std;

__global__
void add_vectors(int* dev_a, int* dev_b, int* dev_c, int size)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
 
    if (tid < size)
    {
        dev_c[tid] = dev_a[tid] + dev_b[tid];
    }
}

void print_vector(int* vec, int size)
{
    for (int i = 0; i < size; ++i)
    {
        cout << vec[i] << " ";
    }
    cout << "\n";
}

int main()
{
    int SIZE = 3;
    int* a = (int*)malloc(SIZE * sizeof(int));
    int* b = (int*)malloc(SIZE * sizeof(int));
    int* c = (int*)malloc(SIZE * sizeof(int));
 
    for (int i = 0; i < SIZE; ++i)
    {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }
 
    cout << "Vector A: ";
    print_vector(a, SIZE);

    cout << "Vector B: ";
    print_vector(b, SIZE);
 
    int* dev_a, *dev_b, *dev_c;

    cudaMalloc(&dev_a, SIZE * sizeof(int));
    cudaMalloc(&dev_b, SIZE * sizeof(int));
    cudaMalloc(&dev_c, SIZE * sizeof(int));
 
    cudaMemcpy(dev_a, a, SIZE * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, SIZE * sizeof(int), cudaMemcpyHostToDevice);
 
    int threadsPerBlock = 256;
    int blocksPerGrid = (SIZE + threadsPerBlock - 1) / threadsPerBlock;
    
    add_vectors<<<blocksPerGrid, threadsPerBlock>>>(dev_a, dev_b, dev_c, SIZE);
 
    cudaMemcpy(c, dev_c, SIZE * sizeof(int), cudaMemcpyDeviceToHost);

    cout << "Vector C: ";
    print_vector(c, SIZE);

    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
 
    free(a);
    free(b);
    free(c);
}

Vector A: 3 7 3 
Vector B: 6 5 5 
Vector C: 9 12 8 

